In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_facenet import FaceNet
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from ultralytics import YOLO  # YOLOv8 library

# Load YOLOv8 model for face detection
face_detector = YOLO("face_yolov8s.pt")
print("YOLOv8 Face Detection Model loaded successfully!")

# Step 1: Define paths
dataset_path = "dataset"
classes = ["Aahad", "Ali", "Baba", "Bisma", "Damysha", "Daud", "Fahad", "Gujjar", "Sameed", "Saqib", "Unaiza", "Zunaira"]

# Step 2: Detect and crop face
def detect_and_crop_face(image_path):
    """
    Detects and crops the face region from an image using YOLOv8 face detection.
    Args:
        image_path (str): Path to the input image.
    Returns:
        PIL.Image: Cropped face image or None if no face is detected.
    """
    image = Image.open(image_path).convert("RGB")
    results = face_detector.predict(source=np.array(image), conf=0.5, iou=0.45, save=False, save_crop=False, device="cpu")
    
    # Process detection results
    if len(results[0].boxes) > 0:
        # Take the first detected face (highest confidence)
        box = results[0].boxes[0].xyxy[0].cpu().numpy()
        x1, y1, x2, y2 = map(int, box)
        cropped_face = image.crop((x1, y1, x2, y2))
        return cropped_face.resize((160, 160))  # Resize for FaceNet
    else:
        return None  # No face detected

# Step 3: Augmentation setup
def augment_images(image, output_dir, augmentations=5):
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    image_array = np.expand_dims(np.array(image), axis=0)
    augmented_images = datagen.flow(image_array, batch_size=1)
    for i in range(augmentations):
        aug_image = next(augmented_images)[0]
        Image.fromarray(aug_image.astype('uint8')).save(os.path.join(output_dir, f"aug_{i}.jpg"))

# Step 4: Prepare the dataset with face detection and augmentation
def prepare_augmented_dataset(dataset_path, augmentations=5):
    data = []
    labels = []
    for label, cls in enumerate(classes):
        class_path = os.path.join(dataset_path, cls)
        aug_output_path = os.path.join(dataset_path, f"{cls}_augmented")
        os.makedirs(aug_output_path, exist_ok=True)

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            cropped_face = detect_and_crop_face(img_path)
            if cropped_face is not None:
                # Original face image
                data.append(np.array(cropped_face))
                labels.append(label)
                # Augmented face images
                augment_images(cropped_face, aug_output_path, augmentations)
                for aug_img_name in os.listdir(aug_output_path):
                    aug_img_path = os.path.join(aug_output_path, aug_img_name)
                    data.append(np.array(Image.open(aug_img_path)))
                    labels.append(label)
    return np.array(data), np.array(labels)

print("Detecting faces, augmenting images, and preparing dataset...")
X, y = prepare_augmented_dataset(dataset_path, augmentations=5)
print(f"Dataset prepared with {len(X)} images.")

# Step 5: Generate embeddings using FaceNet
print("Generating embeddings...")
facenet_model = FaceNet()
embeddings = np.array([facenet_model.embeddings([img])[0] for img in X])
print("Embeddings generated successfully!")

# Step 6: Train/Test split
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)

# Step 7: Train the SVM classifier
print("Training SVM classifier...")
classifier = SVC(kernel='linear', probability=True)
classifier.fit(X_train, y_train)
print("Classifier trained successfully!")

# Step 8: Evaluate the model
y_pred = classifier.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=classes))

# Step 9: Print Accuracy in Percentage
accuracy = accuracy_score(y_test, y_pred) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Step 10: Predict an input image
import cv2

def predict_image(image_path):
    """
    Predicts whether the face in the input image is 'aahad' or 'sameed'.
    Args:
        image_path (str): Path to the input image.
    Returns:
        str: Predicted class label.
    """
    # Detect and crop the face
    cropped_face = detect_and_crop_face(image_path)
    if cropped_face is not None:
        # Convert PIL image to NumPy array
        cropped_face_array = np.array(cropped_face)
        
        # Resize image to FaceNet's expected input size (160x160)
        cropped_face_resized = cv2.resize(cropped_face_array, (160, 160))
        
        # Generate embedding for the cropped face
        embedding = facenet_model.embeddings([cropped_face_resized])[0].reshape(1, -1)
        
        # Predict using the trained SVM classifier
        prediction = classifier.predict(embedding)
        return classes[prediction[0]]
    else:
        return "No face detected"


In [ ]:
import joblib

# Save the trained classifier
joblib.dump(classifier, 'classifier.pkl')
print("Classifier saved successfully!")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Step 8: Evaluate the model
y_pred = classifier.predict(X_test)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=classes))

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()